In [ ]:
# Load the data and libraries
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
from scipy.special import expit as invlogit

data = pd.read_csv('./data/Free throw.csv.zip').drop('Timestamp', axis=1)
data.columns = ['age', 'gender', 'athleticism', 'y0', 'y1']

true_effect = data.y1 - data.y0
y0 = data.y0
y1 = data.y1

n = len(data)

np.random.seed(42)
n = 1000
x = np.random.randint(1, 11, size=n)
y1 = np.random.binomial(50, invlogit(0.2 * x - 1.5))
y0 = np.random.binomial(50, invlogit(0.2 * x - 2.0))
data = pd.DataFrame({"x": x, "y1": y1, "y0": y0})

print(f'True ATE for all respondents {true_effect.mean():.2f} with an sd of {true_effect.std():.2f}')



In [ ]:
### Assign random treatment

z = np.random.binomial(1, 0.5, size=n)
y = np.where(z == 1, y1, y0)

est = np.mean(y[z == 1]) - np.mean(y[z == 0])
s1 = np.std(y[z == 1], ddof=1)
s0 = np.std(y[z == 0], ddof=1)
n1 = np.sum(z == 1)
n0 = np.sum(z == 0)
se = np.sqrt(s1**2 / n1 + s0**2 / n0)

print(f'Random treatment ETA {est:.2f} with sd of {se:.2f}')

In [ ]:
## Next let's estimate it using a model

with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfCauchy("sigma", beta=5)

    mu = alpha + beta * z
    likelihood = pm.Normal("y", mu=mu, sigma=sigma, observed=y)

    trace = pm.sample()

print(pm.summary(trace))
pm.plot_posterior(trace, var_names=["beta"], ref_val=5.66)

In [ ]:
trace.posterior.beta.plot.hist()

In [ ]:
### Biased estimate

p_treat = invlogit(x - 5.5)
z = np.random.binomial(1, p_treat)
y = np.where(z == 1, y1, y0)

# Estimate the ATE with a model

with pm.Model() as model2:
    ...



In [ ]:
### Now add adjustment for pre-treatment and model the ETA again



In [ ]:
# Biased in the other direction

p_treat = invlogit(-(x - 5.5))
z = np.random.binomial(1, p_treat)
y = np.where(z == 1, y1, y0)

# Estimate the ATE with a model

with pm.Model() as model3:
    ...

In [ ]:
# Code the model

import pymc as pm

with pm.Model() as model:
    ...